In [13]:
import cv2
import mediapipe as mp
import random
import time

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

wCam, hCam = 640, 480
cap = cv2.VideoCapture(0) 
cap.set(3, wCam)
cap.set(4, hCam)

with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    pTime = 0

    while True:
        success, img = cap.read()
        if not success:
            break

        img = cv2.flip(img, 1)

        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_img)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

                distances = {
                    'thumb_index': (index_tip.x - thumb_tip.x)**2 + (index_tip.y - thumb_tip.y)**2,
                    'index_middle': (middle_tip.x - index_tip.x)**2 + (middle_tip.y - index_tip.y)**2,
                    'middle_ring': (ring_tip.x - middle_tip.x)**2 + (ring_tip.y - middle_tip.y)**2,
                    'ring_pinky': (pinky_tip.x - ring_tip.x)**2 + (pinky_tip.y - ring_tip.y)**2
                }

                if distances['thumb_index'] > 0.03 and distances['index_middle'] > 0.03 and distances['middle_ring'] > 0.03 and distances['ring_pinky'] > 0.03:
                    gesture = "Paper"
                    cv2.putText(img, "Paper", (40, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                elif distances['thumb_index'] < 0.01 and distances['index_middle'] < 0.01 and distances['middle_ring'] < 0.01 and distances['ring_pinky'] < 0.01:
                    gesture = "Rock"
                    cv2.putText(img, "Rock", (40, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                else:
                    gesture = "Scissors"
                    cv2.putText(img, "Scissors", (40, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

            computer_choice = random.choice(["Rock", "Paper", "Scissors"])

            if gesture == computer_choice:
                result = "It's a Draw!"
            elif (gesture == "Rock" and computer_choice == "Scissors") or \
                 (gesture == "Paper" and computer_choice == "Rock") or \
                 (gesture == "Scissors" and computer_choice == "Paper"):
                result = "You Win!"
            else:
                result = "You Lose!"

            cv2.putText(img, f"Computer: {computer_choice}", (40, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
            cv2.putText(img, result, (40, 550), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow("Rock Paper Scissors Game", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
